In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import pandas as pd
import json
# Any results you write to the current directory are saved as output.

In [55]:
print ("Read Dataset ... ")
def read_dataset(path):
	return json.load(open(path)) 
train = read_dataset('train.json')
# print(len(train))
test = read_dataset('test.json')
train1 = read_dataset('cooking_train.json')

# Text Data Features
print ("Prepare text data of Train and Test ... ")
def generate_text(data):
	text_data = [" ".join(doc['ingredients']).lower() for doc in data]
	return text_data 

train_text = generate_text(train)
train1_text = generate_text(train1)
test_text = generate_text(test)
target1 = [doc['cuisine'] for doc in train]
target2 = [doc['cuisine'] for doc in train1]

for i in range(len(train_text)-1):
    train_text[i] = train_text[i].replace('salt','')

# Feature Engineering 
print ("TF-IDF on text data ... ")
tfidf = TfidfVectorizer(binary=True, norm='l2')
# tfidf = CountVectorizer(binary=True)
def tfidf_features(txt, flag):
    if flag == "train":
    	x = tfidf.fit_transform(txt)
    else:
	    x = tfidf.transform(txt)
    x = x.astype('float')
    return x 

X = tfidf_features(train_text, flag="train")
X_test = tfidf_features(test_text, flag="test")
XX = tfidf_features(train1_text, flag="test")

Read Dataset ... 
Prepare text data of Train and Test ... 
TF-IDF on text data ... 


In [56]:
a = pd.DataFrame(X.toarray())
b = pd.DataFrame(XX.toarray())
c = a.append(b[:])

X_test = pd.DataFrame(X_test.toarray())

In [57]:
len(target1)

39774

In [58]:
target = target1 + target2[:]

In [59]:
print(len(target))
print(len(c))
len(X_test)

69774
69774


9944

In [60]:
# Label Encoding - Target 
print ("Label Encode the Target Variable ... ")
lb = LabelEncoder()
y = lb.fit_transform(target)
# X_scaled = preprocessing.scale(X.toarray())
# test_scaled = preprocessing.scale(X_test.toarray())


Label Encode the Target Variable ... 


In [7]:
# %%time
# from sklearn.decomposition import PCA
# pca = PCA(n_components=0.95)
# X_pca = pca.fit_transform(X.toarray())
# print(pca.n_components_)

In [8]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(c, y , random_state = 0,test_size=0.2)

In [9]:
preds = pd.DataFrame()

In [ ]:
%%time
from sklearn.svm import NuSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
model = SVC(C=100, # penalty parameter, setting it to a larger value 
                 kernel='rbf', # kernel type, rbf working fine here
                 degree=3, # default value, not tuned yet
                 gamma=2, # kernel coefficient, not tuned yet
                 coef0=1, # change to 1 from default value of 0.0
                 shrinking=True, # using shrinking heuristics
                 tol=0.00001, # stopping criterion tolerance 
                 probability=False, # no need to enable probability estimates
                 class_weight=None, # all classes are treated equally 
                 verbose=1, # print the logs 
                 max_iter=-1, # no limit, let it run
                 decision_function_shape=None, # will use one vs rest explicitly 
                 random_state=None)

# model = BaggingClassifier(m2, max_samples=0.5, max_features=0.5)
# model = AdaBoostClassifier(base_estimator=m2,algorithm='SAMME',learning_rate=0.1)
# model = OneVsRestClassifier(classifier, n_jobs=-1)

model.fit(c,y)
preds['svm'] = model.predict(X_test)

[LibSVM]

In [61]:
%%time
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier
m1 = LGBMClassifier(n_estimators=550, max_depth=15)

# model = BaggingClassifier(m1, max_samples=0.8, max_features=0.8)
m1.fit(c[:50000],y[:50000])
y_pred = m1.predict(X_test)



Wall time: 5min 46s


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [62]:
preds['lgbm'] = lb.inverse_transform(y_pred)

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [63]:
op1 = pd.read_csv('svm_output (1).csv')
op2 = pd.read_csv('svm_output_ensemble.csv')
preds['svm1'] = op1.cuisine
preds['svm2'] = op2.cuisine

In [70]:
preds

,cnn,lgbm,svm1,svm2
0,british,irish,irish,irish
1,southern_us,southern_us,southern_us,southern_us
2,italian,italian,italian,italian
3,cajun_creole,cajun_creole,cajun_creole,cajun_creole
4,italian,italian,italian,italian
5,southern_us,southern_us,southern_us,southern_us
6,french,southern_us,greek,italian
7,chinese,chinese,chinese,chinese
8,mexican,mexican,mexican,mexican
9,british,british,irish,british


In [71]:
cols = list(preds)
cols.insert(0,cols.pop(cols.index('svm2')))
preds12 = preds.loc[:,cols]

In [73]:
preds12 = preds12.mode(axis=1)

In [52]:
## DNN Solution
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization, ActivityRegularization
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.cross_validation import train_test_split
Y = to_categorical(y, num_classes=20)
random_seed = 2
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(a, Y[:39774], test_size = 0.4, random_state=random_seed)

X_train = X_train.append(b)
yyy = list(Y_train)
yyy.extend(list(Y[39774:]))
Y_train = np.array(yyy)

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(3051,)))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dense(128, activation = "relu"))
model.add(Dense(128, activation = "relu"))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(20, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=1e-04)
optimizer1 = Adam(lr=0.001, decay=1e-4)
optimizer2 = SGD(lr=0.1, decay=1e-4)
model.compile(optimizer = optimizer1 , loss = "categorical_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=0.01, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
earlystopping = EarlyStopping(monitor='val_acc', patience=0.001, verbose=1, mode='auto')

In [53]:
history = model.fit(X_train, Y_train,
                      epochs=30,
                      batch_size=128,
                      validation_data=(X_val,Y_val),
                      verbose=1,callbacks=[learning_rate_reduction])

Train on 53864 samples, validate on 15910 samples
Epoch 1/35
53864/53864 [==============================] - 4s 81us/step - loss: 1.6116 - acc: 0.5305 - val_loss: 1.0017 - val_acc: 0.7004
Epoch 2/35
53864/53864 [==============================] - 4s 75us/step - loss: 1.0136 - acc: 0.7121 - val_loss: 0.8367 - val_acc: 0.7626
Epoch 3/35
53864/53864 [==============================] - 4s 75us/step - loss: 0.7890 - acc: 0.7762 - val_loss: 0.7064 - val_acc: 0.8077
Epoch 4/35
53864/53864 [==============================] - 4s 73us/step - loss: 0.6299 - acc: 0.8195 - val_loss: 0.6288 - val_acc: 0.8422
Epoch 5/35
53864/53864 [==============================] - 4s 74us/step - loss: 0.4946 - acc: 0.8585 - val_loss: 0.5941 - val_acc: 0.8632
Epoch 6/35
53864/53864 [==============================] - 4s 73us/step - loss: 0.3973 - acc: 0.8874 - val_loss: 0.6035 - val_acc: 0.8723
Epoch 7/35
53864/53864 [==============================] - 4s 74us/step - loss: 0.3279 - acc: 0.9060 - val_loss: 0.5669 - val_acc

In [50]:
preds = pd.DataFrame()
y_pred = model.predict(X_test)

results = np.argmax(y_pred,axis = 1)

preds['cnn'] = lb.inverse_transform(results)

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [76]:
preds12[0]

0              irish
1        southern_us
2            italian
3       cajun_creole
4            italian
5        southern_us
6             french
7            chinese
8            mexican
9            british
10           italian
11             greek
12            indian
13           italian
14           british
15            french
16           mexican
17       southern_us
18           mexican
19       southern_us
20          japanese
21            indian
22            french
23        vietnamese
24           italian
25       southern_us
26              thai
27            korean
28           italian
29       southern_us
            ...     
9914      vietnamese
9915         italian
9916           greek
9917         italian
9918     southern_us
9919       brazilian
9920         italian
9921            thai
9922          indian
9923         italian
9924     southern_us
9925         british
9926     southern_us
9927     southern_us
9928          indian
9929    cajun_creole
9930         

In [77]:
print ("Generate Submission File ... ")
test_id = [doc['id'] for doc in test]
sub = pd.DataFrame({'id': test_id, 'cuisine': preds12[0]}, columns=['id', 'cuisine'])
sub.to_csv('ensemble.csv', index=False)

Generate Submission File ... 
